In [4]:
import numpy as np
import torch 

In [5]:
#Objective Functions and Inequality Constraints
obj = lambda x: x[0]**2 + (x[1] - 3)**2
g1 = lambda x: x[1]**2 - 2*x[0]
g2 = lambda x: (x[1]-1)**2 + 5*x[0]-15
df_obj  = lambda x : torch.tensor([[2*x[0], 2*(x[1]-3)]])
df_g1 = lambda x : torch.tensor([[-2, 2*x[1]]])
df_g2 = lambda x: torch.tensor([[5, 2*(x[1]-1)]])

In [16]:
def PenaltyWeights(mu, weights,k):
  if k > 0:
    weights = torch.max(abs(mu), 0.5 *(weights + abs(mu)))    #For rest of the iterations
  else:   
    weights = abs(mu)     #For first iteration
  return weights

In [17]:
def F_alpha(x,weights,alpha,s):
  G1=max(0, g1(x + alpha*s)) # constraint 1
  G2=max(0, g2(x + alpha*s)) # constraint 2
  return obj(x + alpha*s) +  weights[0,:] * G1 + weights[1,:]* G2 # Merit Function Equation

In [18]:
def ArmijiloLineSearch(x,mu,weights,s,k):
    t = 0.4
    alpha = 1
    weights = PenaltyWeights(mu, weights,k)
    if g1(x) <= 0:
       dg1_da = 0
    else:
      dg1_da= torch.matmul(df_g1(x), s)

    if g2(x) <= 0:
       dg2_da = 0
    else:
      dg2_da= torch.matmul(df_g2(x), s)

    dF_da = torch.matmul(df_obj(x), s) + (weights[0, :]*dg1_da + weights[1, :]*dg2_da)

    phi = lambda x, weights, alpha, t, dF_da: F_alpha(x, weights, 0, 0) + alpha*t*dF_da

    while phi(x, weights, alpha, t, dF_da) < F_alpha(x, weights, alpha, s):
        alpha = 0.5 * alpha
    return alpha, weights

In [9]:
def BFGS(x,W, s, mu, alpha):
  lx_k = df_obj(x) + torch.matmul(mu.T, torch.tensor([g1(x),g2(x)]))
  lx_k_1 = df_obj(x + alpha*s) + torch.matmul(mu.T, torch.tensor([g1(x + alpha*s),g2(x + alpha*s )]))

  delta_l = lx_k_1 -lx_k
  
  Q = torch.matmul(torch.matmul((alpha*s).T, W), (alpha*s))
  if torch.matmul((alpha*s).T, delta_l.T) >= 0.2 * torch.matmul(torch.matmul((alpha*s).T, W), (alpha*s)):
        theta = 1
  else:
        theta = 0.8 * Q / (Q - torch.matmul((alpha*s).T, delta_l.T))
  y = theta * delta_l.T + (1 - theta) * torch.matmul(W, (alpha*s))
  W = W + torch.matmul(y, y.T) / torch.matmul(y.T, s) - torch.matmul(torch.matmul(W, s), torch.matmul(s.T, W)) / torch.matmul(torch.matmul(s.T, W), s)
  return W

In [19]:
def MuCheck(mu,active):
    mu_check = 0
    if len(mu) == 0 or min(mu) > 0:
        mu_check  = 1
    else:
        mu_idx = np.argmin(np.array(mu))
        mu = mu[mu!=min(mu)]
        active.pop(mu_idx)
    return active, mu_check ,mu

In [20]:
def solve_sqp(x, W):
    active = []
    A_0 = torch.cat((df_g1(x),df_g2(x)),0)
    B_0 = torch.tensor([[g1(x), g2(x)]]).T
    mu_0 = torch.zeros((B_0.shape[0], 1))
    mu = []
    while True:
      if len(active) == 0:
            s_mu = torch.matmul(torch.linalg.inv(W), -df_obj(x).T)
            s = s_mu[:2, :]
      if len(active) > 0:
        if len(active) == 1:
                A = A_0[active[0], :].reshape(1, -1)
                B = B_0[active[0], :].reshape(1,1)
        if len(active) == 2:
                A = A_0
                B = B_0
                
        Z = torch.zeros((A.shape[0], A.shape[0]))
        matrix=torch.cat((torch.cat((W,A.T),1),torch.cat((A,Z),1)),0)
        j=torch.cat((-df_obj(x).T,-B),0)
        s_mu = torch.matmul(torch.linalg.inv(matrix), j)
        s = s_mu[:2, :]
        mu = s_mu[2:, :]

      if len(mu) == 1:
          mu_0[0] = s_mu[2:3, :]
      if len(mu) == 2:
          mu_0[0] = s_mu[2:3, :]
          mu_0[1] = s_mu[3:, :]

      sqp_constraint = torch.round((torch.matmul(A_0, s.reshape(-1, 1)) + B_0))
      active, mu_check,mu = MuCheck(mu,active)

      if torch.max(sqp_constraint) <= 0 and mu_check == 1:
            return s, mu_0
      else:
          index = np.argmax(sqp_constraint)
          active.append(index)
          active = np.unique(np.array(active)).tolist()

In [12]:
x = torch.tensor([[1,1.]]).T
x_initial = x
mu = torch.zeros((x.shape[0], 1))
weights = torch.zeros((x.shape[0], 1))+2
W = torch.eye(x.shape[0])
eps = 1e-4
k = 0

delta_L_norm = np.linalg.norm(df_obj(x) + np.matmul(mu.T, torch.cat((df_g1(x),df_g2(x)),0)))

while delta_L_norm > eps:
    s, mu = solve_sqp(x, W)
    a, weights = ArmijiloLineSearch(x,mu,weights,s,k)
    weights_old = weights
    W = BFGS(x,W, s, mu, a) 
    x += a*s
    k += 1
    delta_L_norm = np.linalg.norm(df_obj(x) + np.matmul(mu.T, torch.cat((df_g1(x),df_g2(x)),0)))

In [13]:
print(f'Solution for this problem is: X=({x[0][0]},{x[1][0]})')

Solution for this problem is: X=(1.0602166652679443,1.4561707973480225)


In [15]:
print(f'The minumim value of objective function is {obj([x[0][0],x[1][0]])}')
print(f'The inequality constraints are: \n g1={g1([x[0][0],x[1][0]])} \n g2={g2([x[0][0],x[1][0]])}')

The minumim value of objective function is 3.507467746734619
The inequality constraints are: 
 g1=0.0 
 g2=-9.490824699401855


Both the inequality constraints are satisfied, hence this is a feasible solution, and from SQP algorithm, this solution is the optimal solution.